In [ ]:
# Importing flask module in the project is mandatory
# An object of Flask class is our WSGI application.
from flask import Flask, redirect, url_for, request, render_template, send_file
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from datetime import datetime
import time
import random
import re
import sys
from flask_mysqldb import MySQL

# Flask constructor takes the name of
# current module (__name__) as argument.
app = Flask(__name__)

# initialize database
app.config['MYSQL_HOST'] = 'localhost'  # MySQL server host
app.config['MYSQL_USER'] = 'root'  # MySQL username
app.config['MYSQL_PASSWORD'] = ''  # MySQL password
app.config['MYSQL_DB'] = 'webscrapper_nusatrip'  # MySQL database name

mysql = MySQL(app)

#Initialize data 
data = []
job_id = ""
from_ = ""
to = ""
cabin_class =""
date = ""
time_temp = ""

#scraping function for booking.com
def booking(job_id, from_, to, cabin_class, date):
    options = webdriver.ChromeOptions()
    # options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)
    driver.set_window_size(1920, 1080)  # Set window size for consistent behavior
    driver.get("https://flights.booking.com/flights/"+from_+".AIRPORT-"+to+".AIRPORT/?type=ONEWAY&adults=1&cabinClass="+cabin_class+"&children=&from="+from_+".AIRPORT&to="+to+".AIRPORT&fromCountry=&toCountry=&fromLocationName=Soekarno-Hatta+International+Airport&toLocationName=Changi+Airport&stops=0&depart="+date+"&sort=BEST&travelPurpose=leisure&aid=304142&label=gen173nr-1FEgdmbGlnaHRzKIICOOgHSDNYBGhoiAEBmAExuAEXyAEM2AEB6AEB-AELiAIBqAIDuAL4spmlBsACAdICJDU1MjlmNzk3LTY2ZGYtNDU3ZS05MGQ0LThiNjI5YmUwODA3MNgCBuACAQ")

    c = 0
    current_page_number = 1

    #wait for search button elements visible
    WebDriverWait(driver, .5).until(EC.visibility_of_all_elements_located((By.XPATH, '//button[@data-ui-name="button_search_submit"]')))
    # driver.minimize_window()

    # WebDriverWait(driver, .5).until(EC.element_to_be_clickable((By.XPATH, '//button[@data-ui-name="button_search_submit"]')))

    time.sleep(random.uniform(.2,.5))

    search_button = driver.find_element(By.XPATH, '//button[@data-ui-name="button_search_submit"]')
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//button[@data-ui-name="button_search_submit"]')))
    driver.execute_script("arguments[0].scrollIntoView();", search_button)
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//button[@data-ui-name="button_search_submit"]')))
    WebDriverWait(driver, 10).until(EC.invisibility_of_element_located((By.CSS_SELECTOR, 'div.css-ev1b4i[data-testid="search_loading_overlay"]')))

    search_button.click()

    #wait for list airlines visible
    time.sleep(random.uniform(.5,1.5))

    list_airlines = driver.find_elements(By.XPATH, '//div[@class="css-4o3ibe"]')

    try:
        pagination = driver.find_element(By.CLASS_NAME, 'Pagination-module__pages___wQykF')
        page_buttons = pagination.find_elements(By.TAG_NAME, 'button')
    except Exception as e:
        print(current_page_number,"/",1)
        print((current_page_number/1)*100,"%")
        for i in range(len(list_airlines)):
            
            try:
                c += 1
                flight_details = driver.find_elements(By.CSS_SELECTOR, 'button[data-testid="flight_card_bound_select_flight"]')
                
                if i >= len(flight_details):
                    # Index out of range, break out of the loop
                    break
                
                WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'button[data-testid="flight_card_bound_select_flight"]')))
                # WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[data-testid="flight_card_bound_select_flight"]')))

                time.sleep(random.uniform(.1,.5))
                try:       
                    flight_details[i].click()
                except:
                    search_again_button = driver.find_element(By.XPATH, '//button[@data-testid="Button" and text()="Search again"]')
                    WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, '//button[@data-testid="Button" and text()="Search again"]')))
                    # WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//button[@data-testid="Button" and text()="Search again"]')))
                    search_again_button.click()
                    break

                WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div[data-testid="timeline_leg_info_flight_number_and_class"]')))
                element = driver.find_element(By.CSS_SELECTOR, 'div[data-testid="timeline_leg_info_flight_number_and_class"]')
                text = element.text

                flight_code_element = driver.find_elements(By.XPATH, '//div[@data-testid="timeline_leg_info_flight_number_and_class"]')
                flight_code_list = [element.text.split("·")[0].strip() for element in flight_code_element]
                flight_code_string = ', '.join(flight_code_list)


                # flight_cabin_element = driver.find_elements(By.XPATH, '//div[@data-testid="timeline_leg_info_flight_number_and_class"]')
                flight_cabin_list = [element.text.split("·")[-1].strip() for element in flight_code_element]
                flight_cabin= ', '.join(set(flight_cabin_list))

                departure_elements = driver.find_elements(By.XPATH, '//div[@data-testid="timeline_location_airport_departure"]')
                arrival_elements = driver.find_elements(By.XPATH, '//div[@data-testid="timeline_location_airport_arrival"]')

                total_transit_element = driver.find_elements(By.XPATH, '//div[@data-testid="flight_card_segment_stops_0"]')[i]
                if total_transit_element.text == 'Direct':
                    transit_info= "None"
                else:
                    departure_content = [element.text.split(" · ")[0] for element in departure_elements]
                    arrival_content = [element.text.split(" · ")[0] for element in arrival_elements]
                    truncate_transit_info = departure_content[:-1] + [arrival_content[-2], arrival_content[-1]]
                    transit_info = ", ".join(truncate_transit_info)

                flight_details_close = driver.find_element(By.XPATH, '//button[@aria-label="Close"]')
                WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, '//button[@aria-label="Close"]')))
                # WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Close"]')))

                flight_details_close.click()

                bandara_awal = driver.find_elements(By.XPATH, '//div[@data-testid="flight_card_segment_departure_airport_0"]')[i].text.split()[0]
                bandara_akhir = driver.find_elements(By.XPATH, '//div[@data-testid="flight_card_segment_destination_airport_0"]')[i].text.split()[-1]

                total_transit_element = driver.find_elements(By.XPATH, '//div[@data-testid="flight_card_segment_stops_0"]')[i]
                total_transit = 0 if total_transit_element.text == 'Direct' else int(re.search(r'\d+', total_transit_element.text).group())

                waktu_berangkat = pd.to_datetime(driver.find_elements(By.XPATH, '//div[@data-testid="flight_card_segment_departure_time_0"]')[i].text).time()
                waktu_tiba = pd.to_datetime(driver.find_elements(By.XPATH, '//div[@data-testid="flight_card_segment_destination_time_0"]')[i].text).time()
                total_waktu = ' '.join(driver.find_elements(By.XPATH, '//div[@data-testid="flight_card_segment_duration_0"]')[i].text.split()[:2])

                harga_element = driver.find_elements(By.XPATH, '//div[@class="css-vxcmzt"]')[i]
                harga = float(''.join(filter(lambda x: x.isdigit() or x == ',', harga_element.text)).replace(',', '.'))
                
                # #market flight info
                # res = driver.find_elements(By.XPATH,'//div[contains(@class, "css-17m9lv6")]/div[contains(@class, "Text-module__root--variant-small_1___7N9d2")]')
                # output = [element.text for element in res]
                # truncated = []
                # truncated = output  # Initialize the truncated list with the output list
                # iterate = True
                # truncate_iteration_temp = 0
                # while iterate:
                #     temp = 0
                #     while temp < len(truncated):
                #         if temp+2 < len(truncated) and truncated[temp+1] == ", ":
                #             truncated[temp:temp+3] = [truncated[temp] + ", " + truncated[temp+2]]
                #             temp += 1
                #         else:
                        
                #             temp += 1
                #     if (truncated[-2] == ", "):
                #         iterate = True
                #     else:
                #         iterate = False

                # truncated = [element.split(", operated by")[0].strip() for element in truncated]
                # fligh_marketing = truncated[truncate_iteration_temp]
                # truncate_iteration_temp+=1
                data.append({
                    'job_id': job_id,
                    'platform': 'BOOKING.COM',
                    'departure_date': date,
                    'departure_airport': bandara_awal.upper(),
                    'destination_airport': bandara_akhir.upper(),
                    'marketing_airline' : "NONE",
                    'flight_code': flight_code_string.upper(),
                    'cabin_class': flight_cabin.upper(),
                    'departure_time': waktu_berangkat,
                    'arrival_time': waktu_tiba,
                    'transit': total_transit,
                    'transit_info': transit_info.upper(),
                    'flight_duration': total_waktu.upper(),
                    'rates': harga
                })

                #print(c)
            except Exception as e:
                print(e)
                break
            



    last_page_button = page_buttons[-1]  # Get the last page button
    last_page_label = last_page_button.get_attribute('aria-label').strip()  # Extract the page label
    last_page_number = int(''.join(filter(str.isdigit, last_page_label)))  # Extract the last page number as an integer

    # Loop over each page button
    for page in range(last_page_number):
        print(current_page_number,"/",last_page_number)
        print("{:.0f}%".format((current_page_number / last_page_number) * 100))

        current_page_number +=1
        # Perform scraping on each page
        for i in range(len(list_airlines)):
            try:
                c += 1
                # flight_details = driver.find_elements(By.CSS_SELECTOR, 'button[data-testid="flight_card_bound_select_flight"]')
                
                # if i >= len(flight_details):
                #     # Index out of range, break out of the loop
                #     break
                
                # WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'button[data-testid="flight_card_bound_select_flight"]')))
                # # WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[data-testid="flight_card_bound_select_flight"]')))

                # time.sleep(random.uniform(.1,.2))
                try:    
                    flight_details = driver.find_elements(By.CSS_SELECTOR, 'button[data-testid="flight_card_bound_select_flight"]')
                    
                    if i >= len(flight_details):
                        # Index out of range, break out of the loop
                        break
                    
                    WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'button[data-testid="flight_card_bound_select_flight"]')))
                    # WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[data-testid="flight_card_bound_select_flight"]')))

                    time.sleep(random.uniform(.5,.7))   
                    flight_details[i].click()
                except:
                    search_again_button = driver.find_element(By.XPATH, '//button[@data-testid="Button" and text()="Search again"]')
                    WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, '//button[@data-testid="Button" and text()="Search again"]')))
                    # WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//button[@data-testid="Button" and text()="Search again"]')))
                    search_again_button.click()
                    break
                    

                WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div[data-testid="timeline_leg_info_flight_number_and_class"]')))
                time.sleep(random.uniform(.1,.2))

                element = driver.find_element(By.CSS_SELECTOR, 'div[data-testid="timeline_leg_info_flight_number_and_class"]')
                text = element.text

                #delay for 
                flight_code_element = driver.find_elements(By.XPATH, '//div[@data-testid="timeline_leg_info_flight_number_and_class"]')
                flight_code_list = [element.text.split("·")[0].strip() for element in flight_code_element]
                flight_code_string = ', '.join(flight_code_list)

                #flight_class = text.split(' · ')[1]
                # flight_cabin_element = driver.find_elements(By.XPATH, '//div[@data-testid="timeline_leg_info_flight_number_and_class"]')
                flight_cabin_list = [element.text.split("·")[-1].strip() for element in flight_code_element]
                flight_cabin= ', '.join(set(flight_cabin_list))


                departure_elements = driver.find_elements(By.XPATH, '//div[@data-testid="timeline_location_airport_departure"]')
                arrival_elements = driver.find_elements(By.XPATH, '//div[@data-testid="timeline_location_airport_arrival"]')
                
                total_transit_element = driver.find_elements(By.XPATH, '//div[@data-testid="flight_card_segment_stops_0"]')[i]
                if total_transit_element.text == 'Direct':
                    transit_info= "None"
                else:
                    departure_content = [element.text.split(" · ")[0] for element in departure_elements]
                    arrival_content = [element.text.split(" · ")[0] for element in arrival_elements]
                    truncate_transit_info = departure_content[:-1] + [arrival_content[-2], arrival_content[-1]]
                    transit_info = ", ".join(truncate_transit_info)
                


                flight_details_close = driver.find_element(By.XPATH, '//button[@aria-label="Close"]')
                WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, '//button[@aria-label="Close"]')))
                # WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Close"]')))


                flight_details_close.click()

                bandara_awal = driver.find_elements(By.XPATH, '//div[@data-testid="flight_card_segment_departure_airport_0"]')[i].text.split()[0]
                bandara_akhir = driver.find_elements(By.XPATH, '//div[@data-testid="flight_card_segment_destination_airport_0"]')[i].text.split()[-1]

                total_transit_element = driver.find_elements(By.XPATH, '//div[@data-testid="flight_card_segment_stops_0"]')[i]
                total_transit = 0 if total_transit_element.text == 'Direct' else int(re.search(r'\d+', total_transit_element.text).group())

                waktu_berangkat = pd.to_datetime(driver.find_elements(By.XPATH, '//div[@data-testid="flight_card_segment_departure_time_0"]')[i].text).time()
                waktu_tiba = pd.to_datetime(driver.find_elements(By.XPATH, '//div[@data-testid="flight_card_segment_destination_time_0"]')[i].text).time()
                total_waktu = ' '.join(driver.find_elements(By.XPATH, '//div[@data-testid="flight_card_segment_duration_0"]')[i].text.split()[:2])

                harga_element = driver.find_elements(By.XPATH, '//div[@class="css-vxcmzt"]')[i]
                harga = float(''.join(filter(lambda x: x.isdigit() or x == ',', harga_element.text)).replace(',', '.'))
                
                #market flight info
                # res = driver.find_elements(By.XPATH,'//div[contains(@class, "css-17m9lv6")]/div[contains(@class, "Text-module__root--variant-small_1___7N9d2")]')
                # output = [element.text for element in res]
                # truncated = []
                # truncated = output  # Initialize the truncated list with the output list
                # iterate = True
                # truncate_iteration_temp = 0

                # while iterate:
                #     temp = 0
                #     while temp < len(truncated):
                #         if temp+2 < len(truncated) and truncated[temp+1] == ", ":
                #             truncated[temp:temp+3] = [truncated[temp] + ", " + truncated[temp+2]]
                #             temp += 1
                #         else:
                        
                #             temp += 1
                #     if (truncated[-2] == ", "):
                #         iterate = True
                #     else:
                #         iterate = False
                # truncated = [element.split(", operated by")[0].strip() for element in truncated]
                # fligh_marketing = truncated[truncate_iteration_temp]


                data.append({
                    'job_id': job_id,
                    'platform': 'BOOKING.COM',
                    'departure_date': date,
                    'departure_airport': bandara_awal.upper(),
                    'destination_airport': bandara_akhir.upper(),
                    'marketing_airline' : "NONE",
                    'flight_code': flight_code_string.upper(),
                    'cabin_class': flight_cabin.upper(),
                    'departure_time': waktu_berangkat,
                    'arrival_time': waktu_tiba,
                    'transit': total_transit,
                    'transit_info': transit_info.upper(),
                    'flight_duration': total_waktu.upper(),
                    'rates': harga
                })

            except Exception as e:
                print(e)
                try:
                    search_again_button = driver.find_element(By.XPATH, '//button[@data-testid="Button" and text()="Search again"]')
                    WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, '//button[@data-testid="Button" and text()="Search again"]')))
                    # WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//button[@data-testid="Button" and text()="Search again"]')))

                    search_again_button.click()
                except Exception as e:
                    flight_details_close = driver.find_element(By.XPATH, '//button[@aria-label="Close"]')
                    WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, '//button[@aria-label="Close"]')))
                    # WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Close"]')))

                    flight_details_close.click()
                    break
                break
                

        WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, '//button[@aria-label="Next"]')))
        # WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Next"]')))

        next_button = driver.find_element(By.XPATH, '//button[@aria-label="Next"]')
        time.sleep(random.uniform(.5,.75))
        next_button.click()
        time.sleep(random.uniform(.5,.6))


#function for saving the dataframe to CSV files
def save():
    print("saving the data...")
    # Get the current date and time
    now = datetime.now()

    # Format the date and time as strings
    formatted_date = now.strftime("%Y%m%d-")  # Format: YYYY-MM-DD
    formatted_time = now.strftime("%H%M%S")  # Format: HH:MM:SS

    # Join the date and time strings
    datetime_string = formatted_date + " " + formatted_time


    df = pd.DataFrame(data)
    df.columns = df.columns.str.upper()
    time_temp = "booking.com-" + date + " " + from_ + " to " + to + "-"+ datetime_string + ".csv"
    df.to_csv("booking.com-" + date + " " + from_ + " to " + to + "-"+ datetime_string + ".csv", index=False)
    print("data saved")
    print (time_temp)
    return(time_temp)
    



#route for download the csv file
@app.route('/download', methods=['POST', 'GET'])
def download():   
    if request.method =='GET':
        css_path = url_for('static', filename='styles.css')
        filename = save()
        print("performing download")
        time.sleep(random.uniform(.1,.5))

        return send_file(filename,as_attachment=True, mimetype='text/csv')
    

#route for testing
@app.route('/test', methods=['GET'])
def test():
    css_path = request.args.get('css_path')
    filename = request.args.get('filename')
    return render_template("scrapper_final.html", css_path=css_path, filename=filename)
   

#route for display scrapper tools
@app.route('/', methods=['GET'])
def display():
    css_path = request.args.get('css_path')
    filename = request.args.get('filename')
    return render_template("scrapper.html", css_path=css_path, filename=filename)
   

#route for scrapping the data
@app.route('/scrapper', methods=['POST', 'GET'])
def scrapper():
    if request.method == 'POST':
        
        from_ = request.form.get('from')
        to = request.form.get('to')
        depart = request.form.get('depart')
        cabin = request.form.get('cabin')

        css_path = url_for('static', filename='styles.css')
        print(from_, to, cabin, depart)
        booking(1, from_, to, cabin, depart)
        print("srcapper.html")

        df = pd.DataFrame(data)
        html_table = df.to_html()
        insert_dataframe()
        return render_template("scrapper.html", css_path = css_path, table=html_table)
        
    else:
        from_ = request.args.get('from')
        to = request.args.get('to')
        depart = request.args.get('depart')
        cabin = request.args.get('cabin')
        platform = request.args.get('platform')

        css_path = url_for('static', filename='styles.css')
        print(from_, to, cabin, depart)
        booking(1, from_, to, cabin, depart)
        print("srcapper.html 1")
        insert_dataframe()
        return render_template("scrapper.html", css_path, as_attachment=False)
        # app.config['MYSQL_HOST'] = 'localhost'  # MySQL server host
        # app.config['MYSQL_USER'] = 'root'  # MySQL username
        # app.config['MYSQL_PASSWORD'] = ''  # MySQL password
        # app.config['MYSQL_DB'] = 'webscrapper_nusatrip'  # MySQL database name
        # try:
        #     # Read the DataFrame from a file or any other data source
        #     # df = pd.read_csv('your_dataframe.csv')  # Replace with the path to your DataFrame file

        #     # Get a cursor to execute SQL queries
        #     cursor = mysql.connection.cursor()

        #     # Iterate through each row of the DataFrame and insert into the database
        #     for _, row in df.iterrows():
        #         query = """
        #         INSERT INTO t_flight_rates (JOB_ID, PLATFORM, DEPARTURE_DATE, DEPARTURE_AIRPORT, DESTINATION_AIRPORT, 
        #         MARKETING_AIRLINE, FLIGHT_CODE, CABIN_CLASS, DEPARTURE_TIME, ARRIVAL_TIME, TRANSIT, TRANSIT_INFO, 
        #         FLIGHT_DURATION, RATES) 
        #         VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        #         """
        #         values = (
        #             row['JOB_ID'],
        #             row['PLATFORM'],
        #             row['DEPARTURE_DATE'],
        #             row['DEPARTURE_AIRPORT'],
        #             row['DESTINATION_AIRPORT'],
        #             row['MARKETING_AIRLINE'],
        #             row['FLIGHT_CODE'],
        #             row['CABIN_CLASS'],
        #             row['DEPARTURE_TIME'],
        #             row['ARRIVAL_TIME'],
        #             row['TRANSIT'],
        #             row['TRANSIT_INFO'],
        #             row['FLIGHT_DURATION'],
        #             row['RATES']
        #         )
        #         cursor.execute(query, values)

        #     # Commit the changes
        #     mysql.connection.commit()

        #     cursor.close()

        #     return 'DataFrame inserted into the database successfully.'

        # except Exception as e:
        #     return 'An error occurred: ' + str(e)

# @app.route('/')
# def index():
#     # Create a sample DataFrame
#     df = pd.DataFrame(data)
    
#     # Generate HTML representation of the DataFrame
#     html_table = df.to_html()
    
#     # Render the template and pass the HTML table
#     return render_template('index.html', table=html_table)


@app.route('/cc')
def index():
    try:
        # Check if the MySQL connection is open
        if mysql.connection:
            print("MySQL connection is established.")
        else:
            print("MySQL connection is not established.")

        return 'Check the console or logs for the MySQL connection status.'
    except Exception as e:
        return 'An error occurred: ' + str(e)
    
@app.route('/at')
def print_table_structure():
    try:
        # Get a cursor to execute SQL queries
        cursor = mysql.connection.cursor()

        # Execute SQL query to retrieve table structure
        table_name = 't_flight_rates'  # Replace with your table name
        query = f"DESCRIBE {table_name}"
        cursor.execute(query)

        # Fetch all rows from the query result
        result = cursor.fetchall()

        # Print the table structure
        for row in result:
            print(row)

        cursor.close()

        return 'Table structure printed in the console or logs.'

    except Exception as e:
        return 'An error occurred: ' + str(e)

@app.route('/adb')
def print_databases_and_tables():
    try:
        # Get a cursor to execute SQL queries
        cursor = mysql.connection.cursor()

        # Execute SQL query to retrieve all databases
        cursor.execute("SHOW DATABASES")

        # Fetch all rows from the query result
        databases = cursor.fetchall()

        # Print all databases
        for db in databases:
            db_name = db[0]
            print(f"Database: {db_name}")

            # Switch to the database
            cursor.execute(f"USE {db_name}")

            # Execute SQL query to retrieve all tables in the current database
            cursor.execute("SHOW TABLES")

            # Fetch all rows from the query result
            tables = cursor.fetchall()

            # Print all tables
            for table in tables:
                table_name = table[0]
                print(f"\tTable: {table_name}")

        cursor.close()

        return 'Database and table information printed in the console or logs.'

    except Exception as e:
        return 'An error occurred: ' + str(e)
    
@app.route('/insert')
def insert_dataframe():
    df = pd.DataFrame(data)
    df.columns = df.columns.str.upper()

    app.config['MYSQL_HOST'] = 'localhost'  # MySQL server host
    app.config['MYSQL_USER'] = 'root'  # MySQL username
    app.config['MYSQL_PASSWORD'] = ''  # MySQL password
    app.config['MYSQL_DB'] = 'webscrapper_nusatrip'  # MySQL database name
    try:
        # Read the DataFrame from a file or any other data source
        # df = pd.read_csv('your_dataframe.csv')  # Replace with the path to your DataFrame file

        # Get a cursor to execute SQL queries
        cursor = mysql.connection.cursor()

        # Iterate through each row of the DataFrame and insert into the database
        for _, row in df.iterrows():
            query = """
            INSERT INTO t_flight_rates (JOB_ID, PLATFORM, DEPARTURE_DATE, DEPARTURE_AIRPORT, DESTINATION_AIRPORT, 
            MARKETING_AIRLINE, FLIGHT_CODE, CABIN_CLASS, DEPARTURE_TIME, ARRIVAL_TIME, TRANSIT, TRANSIT_INFO, 
            FLIGHT_DURATION, RATES) 
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
            values = (
                row['JOB_ID'],
                row['PLATFORM'],
                row['DEPARTURE_DATE'],
                row['DEPARTURE_AIRPORT'],
                row['DESTINATION_AIRPORT'],
                row['MARKETING_AIRLINE'],
                row['FLIGHT_CODE'],
                row['CABIN_CLASS'],
                row['DEPARTURE_TIME'],
                row['ARRIVAL_TIME'],
                row['TRANSIT'],
                row['TRANSIT_INFO'],
                row['FLIGHT_DURATION'],
                row['RATES']
            )
            cursor.execute(query, values)

        # Commit the changes
        mysql.connection.commit()

        cursor.close()

        return 'DataFrame inserted into the database successfully.'

    except Exception as e:
        return 'An error occurred: ' + str(e)
# main driver function
if __name__ == '__main__':
	app.run()


In [ ]:
df_sky = pd.DataFrame({'from':['DPS','SIN'], 'to':['SUB','DPS']})
df_sky

In [ ]:
df_all_nusa = []
df_all = []
for i,j in df_sky[:100].iterrows():
    data = []
    date = '230808'
    job_id = 9
    departure = j['from']
    destination = j['to']
    list_cabin = [2]
    # data_nusa = get_dataNusa(job_id=9, date='8-8-2023', departure=departure, destination=destination, vendor=False, airline_name=True, curr='idr')
    # df_from_nusa = pd.DataFrame(data_nusa)
    # df_all_nusa.append(df_from_nusa)
    df_from_nusa = df_nusa.copy()
    for cabin in list_cabin:
        driver = webdriver.Firefox(options=options)
        cabin_dict = {'0':'economy', '1':'premium_economy', '2':'business', '3':'first'}
        cabin = str(cabin)
        for a,b in cabin_dict.items():
            cabin = cabin.replace(a,b)
        time.sleep(2)
        url= f"https://www.skyscanner.com/transport/flights"
        url= f"{url}/{departure.lower()}/{destination.lower()}/{date}/?adults=1&cabinclass={cabin}&currency=IDR&children=0&qp_prevScreen=HOMEPAGE&ref=home&rtn=0"
        driver.get(url)
        time.sleep(random.uniform(3,6))
        try:
            list_result = driver.find_elements(By.XPATH, '//button[@class="BpkButtonBase_bpk-button_ZGM2M TicketStub_ctaButton_OTNjO"]')
            total = len(list_result)
            if total == 0:
                print("\nBLOCKED BY CAPTCHA")
                time.sleep(65)
            else:
                total = total if total < 5 else 5
                k = total if total < 4 else 3
                total = random.choices(range(total), k=k)
                for num in total:
                    list_result = driver.find_elements(By.XPATH, '//button[@class="BpkButtonBase_bpk-button_ZGM2M TicketStub_ctaButton_OTNjO"]')
                    list_result[num].click()
                    time.sleep(random.uniform(2,4))
                   
                    try:
                        btn_back = driver.find_element(By.XPATH, '//button[@aria-label="back"]')
                        btn_back.click()
                        time.sleep(random.uniform(2,4))
                    except:
                        parent = driver.window_handles[0]
                        chld = driver.window_handles[1]
                        driver.switch_to.window(chld)
                        driver.close()
                        driver.switch_to.window(parent)
                    
                last_height = driver.execute_script("return document.body.scrollHeight")
                scheight = 300
                while scheight < last_height - 500:
                    scheight += random.choice([50,75,100,125])
                    driver.execute_script(f"window.scrollTo(0, {scheight});")
                    time.sleep(random.uniform(.5,2))
        
                time.sleep(random.uniform(8,12))   
                list_result = driver.find_elements(By.XPATH, '//div[@class="EcoTicketWrapper_itineraryContainer__ZWE4O"]')
                for i in list_result:
                    try:
                        maskapai = i.find_element(By.CLASS_NAME, 'BpkImage_bpk-image_img_MDZkN').get_attribute("alt")
                    except:
                        maskapai = i.text.split('\n')[1]
                    maskapai = ','.join(maskapai.split('+'))
                    elem = i.find_element(By.CLASS_NAME, 'LegInfo_legInfo__ZGMzY').text.split('\n')
                    jam = [i for i in elem if ":" in i]
                    berangkat = pd.to_datetime(jam[0].strip()).time()
                    tiba = pd.to_datetime(jam[-1].strip()).time()
                    durasi = elem[2]
                    transit = [i for i in elem[3] if i.isdigit()]
                    transit = int(transit[0]) if len(transit) > 0 else 0
                    transit_info = '-'.join([i for i in elem if len(i.strip()) == 3 and i.isupper()])
                    kode = []
                    try:
                        kode = df_from_nusa[(df_from_nusa['DEPARTURE_AIRPORT'] == departure) & 
                                            (df_from_nusa['DESTINATION_AIRPORT'] == destination) & 
                                            (df_from_nusa['DEPARTURE_TIME'] == berangkat) &
                                            (df_from_nusa['ARRIVAL_TIME'] == tiba)
                                            (df_from_nusa['TRANSIT'] == transit)]['FLIGHT_CODE'].values[0]

                    except:
                        kode = maskapai
                    harga = i.find_element(By.CLASS_NAME, 'Price_mainPriceContainer__MDM3O').text
                    harga = float(''.join([i for i in harga if i.isdigit()]))
                    data.append({
                        "job_id": job_id,
                        "platform": "Skyscanner",
                        "date": pd.to_datetime(date, yearfirst=True).date(),
                        "departure_airport": departure,
                        "destination_airport": destination,
                        "marketing_airline": maskapai,
                        "flight_code": kode,
                        "cabin_class": ' '.join(cabin.split('_')).upper(),
                        "departure_time": berangkat,
                        "arrival_time": tiba,
                        "transit": transit,
                        "transit_info": transit_info,
                        "flight_duration": durasi+'m',
                        "rates": harga
                    })

        except Exception as e:
            print(e)
            time.sleep(15)
            pass
            
        driver.quit()
        time.sleep(random.uniform(4,7)) 
    df = pd.DataFrame(data)
    if len(data) > 0:
        df.to_csv(f'hasil_scrape/{departure}-{destination} {date}.csv', index=False)
    df_all.append(df)
    time.sleep(random.randint(5,9))
    print(f'{departure}-{destination}: {len(data)}', end = ' | ')

# driver.quit()